In [10]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import json

from torch import optim
from param_search import train
from network import GPCopulaRNN
from config import DATA_PATH, e_dict
from utils import transform, inv_transform

In [3]:
#TODO add mean and covariance to existing RP strategy and do the plotting

In [4]:
with open('/Users/mac/Desktop/PycharmProjects/TAADL/models/hyperparams.json', 'r') as file:
    hyp_params = json.load(file)

In [5]:
# import feature 
prices = pd.read_csv(DATA_PATH+'/prices.csv', index_col='Date')

# preprocess weekly data
prices.index = pd.to_datetime(prices.index)
prices = prices.resample('M').last()
lret_m = np.log(prices/prices.shift(1)).fillna(0.0) # weekly return


# convert the data to torch tensor
split_idx = int(lret_m.shape[0] * 0.7)
Z_tr, Z_te = torch.Tensor(lret_m.iloc[:split_idx].values), torch.Tensor(lret_m.iloc[split_idx:].values)


In [8]:
num_samples = 20
num_epochs = 250

# load parameters for model and optimizer
model = GPCopulaRNN(input_size=1, hidden_size=4, num_layers=2, rank_size=4, 
                        batch_size=3, num_asset=7, dropout=0.05, features=e_dict)

optimizer = optim.Adam(params=model.parameters(), lr=hyp_params['learning rate'], weight_decay=hyp_params['weight decay'])
#model.load_state_dict(torch.load('/Users/mac/Desktop/PycharmProjects/TAADL/models/GaussCopula.json')['networks_params'])

In [9]:
# re-train the model
train(model, Z_tr, optimizer, num_samples, num_epochs, save=False)

Epoch 1
Gaussian LL Loss : 41.22999954223633
Validation MSE   : 0.014919459819793701 

Epoch 51
Gaussian LL Loss : -36.619998931884766
Validation MSE   : 0.01605803146958351 

Epoch 101
Gaussian LL Loss : -22.25
Validation MSE   : 0.01620236411690712 

Epoch 151
Gaussian LL Loss : -51.709999084472656
Validation MSE   : 0.013248706236481667 

Epoch 201
Gaussian LL Loss : -35.0099983215332
Validation MSE   : 0.013930179178714752 



tensor(0.0157)

In [ ]:

def test(model, data, optimizer, num_samples, num_epochs, cdf):
    model.init_weight()

    # transform 
    X_te, _ = transform(data, cdf)
    sampler = train_idx_sampler(tr_idx=data.size(0), context_len=12, 
                                    prediction_len=1, num_samples=num_samples)
        
    for epoch in range(num_epochs):
        # randomly sample sequence index for training
        losses = []
        losses_valid = torch.Tensor([0.0])
        for tr_idx, te_idx in sampler: # for each sequence sample
            optimizer.zero_grad()

            # run the model
            mu_t, cov_t, batch_idx = model(data[tr_idx], pred=True)
            x = X_tr[tr_idx][:,batch_idx]

            
            # append loss of each batch
            losses.append(loss.detach())

            # prediction step
            mu_pred, _, _ = model(data[te_idx], pred=True)
            Z_tr_hat = inv_transform(mu_pred.detach(), cdfs)
            
            # calculate validation loss
            losses_valid = losses_valid + (Z_tr_hat[0]- data[te_idx]).pow(2).sum()

        losses_valid = losses_valid[0]/num_samples
        if epoch % 50 == 0:
            print(f'Epoch {epoch+1}') 
            print(f'Gaussian LL Loss : {np.round(np.mean(losses),2)}')
            print(f'Validation MSE   : {losses_valid} \n')
    if save:
        torch.save({'networks_params':model.state_dict(), 'optimizer_dict':optimizer.state_dict()}, 'models/GaussCopula.json')
    
    return losses_valid